In [1]:
import pandas as pd
import itertools as it
import numpy as np

In [2]:
data = pd.read_csv('../Original_Data/annual_generation_source_state.csv')

In [3]:
data = data.drop(['Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9'],axis = 1)

In [4]:
data = data[data.STATE != '  ']
data = data[data.STATE != 'US-TOTAL']
data = data[data.STATE != 'US-Total']
data = data.set_index(np.arange(data.shape[0]))

In [5]:
data.head()

,YEAR,STATE,ENERGY SOURCE,GENERATION (Megawatthours)
0,1990,AK,Total,"5,599,506"
1,1990,AK,Coal,"510,573"
2,1990,AK,Hydroelectric Conventional,"974,521"
3,1990,AK,Natural Gas,"3,466,261"
4,1990,AK,Petroleum,"497,116"


In [6]:
yearlist = []
yearcount = []
for key, group in it.groupby(data.YEAR):
    yearlist.append(key)
    yearcount.append(len(list(group)))
print(yearlist,yearcount)

[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015] [1384, 1410, 1440, 1458, 1481, 1502, 1511, 1527, 1544, 1547, 1590, 1645, 1707, 1712, 1728, 1730, 1743, 1756, 1803, 1819, 1861, 1877, 1915, 1946, 1984, 1976]


In [7]:
statelist = []
for key, group in it.groupby(sorted(data.STATE)):
    statelist.append(key)
print(statelist)

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [8]:
esourcelist = []
for key, group in it.groupby(sorted(data['ENERGY SOURCE'])):
    esourcelist.append(key)
print(esourcelist)

['Coal', 'Geothermal', 'Hydroelectric Conventional', 'Natural Gas', 'Nuclear', 'Other', 'Other Biomass', 'Other Gases', 'Petroleum', 'Pumped Storage', 'Solar Thermal and Photovoltaic', 'Total', 'Wind', 'Wood and Wood Derived Fuels']


In [9]:
arrangedata = pd.DataFrame(index = np.arange(len(yearlist)*len(statelist)),columns = (['Year','State'] + esourcelist))
year = []
for i in yearlist:
    year += list(np.full(len(statelist),i,dtype = int))
arrangedata.Year = year
state = []
for i in range(len(yearlist)):
    state += statelist
arrangedata.State = state

In [10]:
arrangedata.head()

,Year,State,Coal,Geothermal,Hydroelectric Conventional,Natural Gas,Nuclear,Other,Other Biomass,Other Gases,Petroleum,Pumped Storage,Solar Thermal and Photovoltaic,Total,Wind,Wood and Wood Derived Fuels
0,1990,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990,AR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def sumby_asource(source):
    escountlist = []
    for i in range(len(yearlist)):
        for j in statelist:
            esourcesum = 0
            for index, row in data[sum(yearcount[0:i]):sum(yearcount[0:i+1])].iterrows():
                if row['STATE'] == j and row['ENERGY SOURCE'] == source:
                    esourcesum += int(row['GENERATION (Megawatthours)'].replace(',',''))
            escountlist.append(esourcesum)
    return escountlist

In [12]:
for i in esourcelist:
    arrangedata[i] = sumby_asource(i)

In [13]:
arrangedata.to_csv('arranged_esource')

In [14]:
arrangedata.head(10)

,Year,State,Coal,Geothermal,Hydroelectric Conventional,Natural Gas,Nuclear,Other,Other Biomass,Other Gases,Petroleum,Pumped Storage,Solar Thermal and Photovoltaic,Total,Wind,Wood and Wood Derived Fuels
0,1990,AK,1021146,0,1949042,6932522,0,0,0,0,994232,0,0,11199012,0,302070
1,1990,AL,107316230,0,20733014,2041428,24103764,0,95006,538952,276178,0,0,159304266,0,4199694
2,1990,AR,38415870,0,7309306,7157146,22564106,0,30778,0,159958,85944,0,78199196,0,2476088
3,1990,AZ,63831220,0,14835152,4667800,41195378,0,0,0,303734,499534,0,125548594,0,215776
4,1990,CA,5275354,29042508,47585134,148336616,65385614,0,4235830,4293484,10947704,1972504,733336,331569818,5517762,8243972
5,1990,CO,59629966,0,2839740,2580184,0,0,57980,0,54780,-66396,0,65096254,0,0
6,1990,CT,7189290,0,1141238,2502958,39552500,0,2146620,8,17748932,12948,0,70294494,0,0
7,1990,DC,0,0,0,0,0,0,0,0,722086,0,0,722086,0,0
8,1990,DE,10226958,0,0,1526038,0,12630,0,329762,3587612,0,0,15683000,0,0
9,1990,FL,120200514,0,349208,37285366,43559120,2906204,3346156,0,51070276,0,0,263005106,0,4288262
